<div style="text-align: center;">
  <h1 id="model-deployment" style="background-color: #003399; color: white; padding: 10px 20px; margin-bottom: 20px; border-radius: 10px; font-weight: bold;">Model Deployment </h1>
</div>

In this context, model deployment refers to **testing the trained machine learning model on new, unseen data that follows the same format as the raw data in the original dataset**. The goal is to see how well the model can generalize and make accurate predictions when presented with fresh input data. odel deployment in this context essentially tests how well the trained model adapts to new, similar data, ensuring it can still make accurate and reliable predictions outside of the training set.

<div style="text-align: left;">
  <h2 id="libraries-deploy" style="background-color: #003399; color: white; padding: 10px 20px; margin-bottom: 20px; border-radius: 8px; font-weight: bold; display: inline-block;"> 1. Libraries </h2>
</div>

<div style='width: 100%; height: 0.05px; background-color: #003399; border-radius: 10px; margin-top: 20px; margin-bottom: 20px;'></div>

The libraries commonly used for model deployment and prediction tasks include:

In [1]:
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, RobustScaler
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import KNNImputer
from category_encoders import BinaryEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn import set_config
import pickle

<div style="text-align: left;">
  <h2 id="Cleaning" style="background-color: #003399; color: white; padding: 10px 20px; margin-bottom: 20px; border-radius: 8px; font-weight: bold; display: inline-block;"> 2. Cleaning </h2>
</div>

<div style='width: 100%; height: 0.05px; background-color: #003399; border-radius: 10px; margin-top: 20px; margin-bottom: 20px;'></div>

To ensure that new data is properly aligned with the machine learning model that has already been trained and saved, several data cleaning and preprocessing steps need to be performed. These steps are essential to ensure the model can make accurate predictions based on new input data that follows the same format as the original training data

<div style="text-align: left;">
  <h3 style="background-color: #003399; color: white; padding: 10px 20px; margin-bottom: 20px; border-radius: 8px; font-weight: bold; display: inline-block;"> Duplicates </h3>
</div>
drop duplicate data in a dataset

In [2]:
class DropDuplicatesTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, subset=None, keep='first'):
        self.subset = subset
        self.keep = keep

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X.drop_duplicates(subset=self.subset, keep=self.keep).reset_index(drop=True)

<div style="text-align: left;">
  <h3 style="background-color: #003399; color: white; padding: 10px 20px; margin-bottom: 20px; border-radius: 8px; font-weight: bold; display: inline-block;"> Missing Values </h3>
</div>
Handling missing or unknown values consistently between the original and new datasets is crucial for maintaining the accuracy and reliability of the model.

In [3]:
def missing_value_marital(df):
    df = df.dropna(subset=["marital"])
    return df

In [4]:
def handle_missing_job(df):
    job_encoder = LabelEncoder()
    education_encoder = LabelEncoder()

    df['job_encoded'] = job_encoder.fit_transform(df['job'].dropna())
    df['education_encoded'] = education_encoder.fit_transform(df['education'])

    train_data = df[df['job'].notnull()]
    test_data = df[df['job'].isnull()]

    if not train_data.empty:
        X = train_data[['education_encoded']]
        y = train_data['job_encoded']
        model = LogisticRegression(max_iter=200)
        model.fit(X, y)

        if not test_data.empty:
            X_missing = test_data[['education_encoded']]
            predicted_jobs = model.predict(X_missing)
            predicted_job_labels = job_encoder.inverse_transform(predicted_jobs)
            df.loc[df['job'].isnull(), 'job'] = predicted_job_labels

    df.drop(['job_encoded', 'education_encoded'], axis=1, inplace=True)

    return df

In [5]:
def handle_missing_education(df):
    job_encoder = LabelEncoder()
    education_encoder = LabelEncoder()

    df['job_encoded'] = job_encoder.fit_transform(df['job'])
    df['education_encoded'] = education_encoder.fit_transform(df['education'])

    train_data = df[df['education'].notnull()]
    test_data = df[df['education'].isnull()]

    if test_data.empty:
        df.drop(['job_encoded', 'education_encoded'], axis=1, inplace=True, errors='ignore')
        return df

    X_train = train_data[['job_encoded']]
    y_train = train_data['education_encoded']
    model = LogisticRegression(max_iter=200)
    model.fit(X_train, y_train)

    X_test = test_data[['job_encoded']]
    if not X_test.empty:  # Ensure X_test is not empty
        predicted_education = model.predict(X_test)
        predicted_education_labels = education_encoder.inverse_transform(predicted_education.astype(int))
        df.loc[df['education'].isnull(), 'education'] = predicted_education_labels

    df.drop(['job_encoded', 'education_encoded'], axis=1, inplace=True, errors='ignore')

    return df

In [6]:
def handle_missing_default(df):
    # Initialize label encoders
    default_encoder = LabelEncoder()
    job_encoder = LabelEncoder()
    housing_encoder = LabelEncoder()
    loan_encoder = LabelEncoder()

    df['default_encoded'] = df['default'].apply(
        lambda x: default_encoder.fit(df['default'].dropna()).transform([x])[0] if pd.notnull(x) else np.nan
    )
    df['job_encoded'] = job_encoder.fit_transform(df['job'])
    df['housing_encoded'] = housing_encoder.fit_transform(df['housing'])
    df['loan_encoded'] = loan_encoder.fit_transform(df['loan'])

    train_data = df[df['default_encoded'].notna()]
    test_data = df[df['default_encoded'].isna()]

    if test_data.empty:
        df.drop(['default_encoded', 'job_encoded', 'housing_encoded', 'loan_encoded'], axis=1, inplace=True, errors='ignore')
        return df

    X_train = train_data[['job_encoded', 'housing_encoded', 'loan_encoded']]
    y_train = train_data['default_encoded'].astype(int)
    model = RandomForestClassifier(class_weight='balanced', random_state=42)
    model.fit(X_train, y_train)

    X_test = test_data[['job_encoded', 'housing_encoded', 'loan_encoded']]
    if not X_test.empty:  # Ensure X_test is not empty
        predictions = model.predict(X_test)
        test_data['default'] = default_encoder.inverse_transform(predictions.astype(int))
        df.loc[df['default'].isnull(), 'default'] = test_data['default']

    df.drop(['default_encoded', 'job_encoded', 'housing_encoded', 'loan_encoded'], axis=1, inplace=True, errors='ignore')

    return df

In [7]:
def handle_missing_housing_and_loan(df):
    housing_mapping = {'yes': 1, 'no': 0}
    loan_mapping = {'yes': 1, 'no': 0}

    df['housing_encoded'] = df['housing'].map(housing_mapping)
    df['loan_encoded'] = df['loan'].map(loan_mapping)

    job_encoder = {job: idx for idx, job in enumerate(df['job'].unique())}
    df['job_encoded'] = df['job'].map(job_encoder)

    imputer = KNNImputer(n_neighbors=3)
    columns_to_impute = ['housing_encoded', 'loan_encoded', 'job_encoded']
    df[columns_to_impute] = imputer.fit_transform(df[columns_to_impute])

    reverse_housing_mapping = {1: 'yes', 0: 'no'}
    reverse_loan_mapping = {1: 'yes', 0: 'no'}
    reverse_job_encoder = {v: k for k, v in job_encoder.items()}

    df['housing'] = df['housing_encoded'].round().map(reverse_housing_mapping)
    df['loan'] = df['loan_encoded'].round().map(reverse_loan_mapping)
    df['job'] = df['job_encoded'].round().map(reverse_job_encoder)
    df.drop(['housing_encoded', 'loan_encoded', 'job_encoded'], axis=1, inplace=True)

    return df

In [8]:
class MissingValueHandler:
    def fit(self, df, y=None):
        return self

    def transform(self, df):
        df = handle_missing_job(df)
        df = handle_missing_education(df)
        df = handle_missing_default(df)
        df = handle_missing_housing_and_loan(df)
        return df

<div style="text-align: left;">
  <h3 style="background-color: #003399; color: white; padding: 10px 20px; margin-bottom: 20px; border-radius: 8px; font-weight: bold; display: inline-block;"> Binning </h3>
</div>

Binning on three numerical columns (`duration`, `pdays`, `campaign`) can be performed to reduce the impact of outliers and make the data easier to interpret for the model. This technique is particularly beneficial in structuring the data in a way that is more understandable for machine learning algorithms, helping to improve model performance and making the dataset more interpretable.

By converting continuous numerical features into discrete categories, binning can smooth out the effects of extreme values (outliers), reducing their influence on the model. Additionally, it simplifies the relationships between features and the target variable, making the data more structured.

In [9]:
class BinningTransformer:
    def fit(self, df, y=None):
        return self

    def transform(self, df):
        df = df.copy()

        if "duration" in df.columns:
            duration_binned = [0, 60, 180, 300, float("inf")]
            labels_duration = ["Short", "Moderate", "Long", "Very Long"]
            df['duration'] = pd.cut(
                df["duration"], 
                bins=duration_binned, 
                labels=labels_duration, 
                include_lowest=True
            )

        if "pdays" in df.columns:
            df['contacted'] = df['pdays'].apply(
                lambda x: 'Never' if x == 999 else 'Yes' if pd.notnull(x) else np.nan
            )

        if "campaign" in df.columns:
            bins_campaign = [0, 5, 10, 20, 50, float("inf")]
            labels_campaign = ["Hardly ever", "Rarely", "Occasionally", "Often", "Very Often"]
            df['campaign_freq'] = pd.cut(
                df["campaign"], 
                bins=bins_campaign, 
                labels=labels_campaign, 
                include_lowest=True
            )

        return df

<div style="text-align: left;">
  <h3 style="background-color: #003399; color: white; padding: 10px 20px; margin-bottom: 20px; border-radius: 8px; font-weight: bold; display: inline-block;"> Drop Column </h3>
</div>

Removing columns with high VIF or high multicollinearity is an effective strategy to enhance the performance and stability of machine learning models. By reducing redundancy and focusing on the most informative features, can build a more reliable, efficient, and interpretable model that generalizes better to new data. This process helps mitigate potential issues such as overfitting, improves model training time, and makes the final predictions more robust.

In [10]:
columns_to_drop = ['euribor3m', 'nr.employed', 'cons.price.idx', 'poutcome']

In [11]:
with open('best_model.pkl', 'rb') as file:
    final_model = pickle.load(file)

print("Final model loaded successfully!")

Final model loaded successfully!


In [12]:
class DropColumnsTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, columns_to_drop):
        self.columns_to_drop = columns_to_drop

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X = X.copy()
        result = X.drop(columns=self.columns_to_drop, errors='ignore')
        if isinstance(result, pd.Series):
            result = result.to_frame()  # Convert Series to DataFrame
        return result

<div style="text-align: left;">
  <h2 id="load-new-data" style="background-color: #003399; color: white; padding: 10px 20px; margin-bottom: 20px; border-radius: 8px; font-weight: bold; display: inline-block;"> 3. Load New Data </h2>
</div>

<div style='width: 100%; height: 0.05px; background-color: #003399; border-radius: 10px; margin-top: 20px; margin-bottom: 20px;'></div>

Uploading new data.
To simulate a more realistic scenario for model deployment, we generated a new dataset of 10,000 records using libraries such as Faker, Random, and Numpy. This generated data will help us mimic real-world conditions, where the volume of data in the field is likely to be much larger, often reaching thousands of records. By creating this synthetic data, we can evaluate the model’s predictions and performance under conditions that more closely resemble the actual data the model will encounter in production. This will allow us to assess how the model scales with larger datasets and ensure that it can handle the volume and variability of real-world data effectively.

In [13]:
df = pd.read_csv('synthetic_data.csv')
# df = df.drop(columns=["y"])
df

,index,age,job,marital,education,default,housing,loan,contact,month,...,duration,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed
0,0,38,student,married,basic.4y,yes,no,no,cellular,nov,...,2345,44,999,2,nonexistent,-3.383493,94.675099,-44.773086,1.771630,5228.1
1,1,50,admin.,unknown,professional.course,yes,yes,yes,telephone,jun,...,1587,24,999,1,nonexistent,1.387145,94.454781,-39.269555,4.326127,4963.6
2,2,31,technician,single,professional.course,no,unknown,yes,telephone,may,...,308,11,999,6,nonexistent,-1.085762,93.327207,-32.444785,3.045997,4963.6
3,3,63,self-employed,married,professional.course,no,no,unknown,cellular,mar,...,2926,17,999,5,success,-1.814845,92.641923,-43.689510,3.226152,4963.6
4,4,66,management,single,basic.6y,yes,unknown,unknown,cellular,dec,...,2455,28,999,0,nonexistent,-2.501094,94.012726,-37.476266,4.899510,4963.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9995,56,blue-collar,single,basic.9y,no,no,yes,telephone,jul,...,4358,7,1,6,failure,-2.759690,92.876536,-37.399041,4.231601,4991.6
9996,9996,66,services,single,basic.4y,no,unknown,yes,telephone,oct,...,3949,33,1,0,failure,0.191621,92.223093,-40.636561,0.711864,4991.6
9997,9997,28,blue-collar,unknown,basic.4y,no,unknown,unknown,telephone,oct,...,3163,38,19,3,failure,-1.307462,93.518372,-40.654174,4.867393,5017.5
9998,9998,33,technician,divorced,basic.9y,no,yes,yes,telephone,oct,...,2076,31,18,4,nonexistent,0.657740,94.286183,-27.570796,3.422417,5008.7


Converting the index into a column

In [14]:
df = df.set_index("index")
df

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,duration,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed
index,,,,,,,,,,,,,,,,,,,,
0,38,student,married,basic.4y,yes,no,no,cellular,nov,mon,2345,44,999,2,nonexistent,-3.383493,94.675099,-44.773086,1.771630,5228.1
1,50,admin.,unknown,professional.course,yes,yes,yes,telephone,jun,fri,1587,24,999,1,nonexistent,1.387145,94.454781,-39.269555,4.326127,4963.6
2,31,technician,single,professional.course,no,unknown,yes,telephone,may,thu,308,11,999,6,nonexistent,-1.085762,93.327207,-32.444785,3.045997,4963.6
3,63,self-employed,married,professional.course,no,no,unknown,cellular,mar,tue,2926,17,999,5,success,-1.814845,92.641923,-43.689510,3.226152,4963.6
4,66,management,single,basic.6y,yes,unknown,unknown,cellular,dec,tue,2455,28,999,0,nonexistent,-2.501094,94.012726,-37.476266,4.899510,4963.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,56,blue-collar,single,basic.9y,no,no,yes,telephone,jul,thu,4358,7,1,6,failure,-2.759690,92.876536,-37.399041,4.231601,4991.6
9996,66,services,single,basic.4y,no,unknown,yes,telephone,oct,thu,3949,33,1,0,failure,0.191621,92.223093,-40.636561,0.711864,4991.6
9997,28,blue-collar,unknown,basic.4y,no,unknown,unknown,telephone,oct,tue,3163,38,19,3,failure,-1.307462,93.518372,-40.654174,4.867393,5017.5


<div style="text-align: left;">
  <h2 id="predict" style="background-color: #003399; color: white; padding: 10px 20px; margin-bottom: 20px; border-radius: 8px; font-weight: bold; display: inline-block;"> 4. Predict </h2>
</div>

<div style='width: 100%; height: 0.05px; background-color: #003399; border-radius: 10px; margin-top: 20px; margin-bottom: 20px;'></div>

Performing the data prediction process.

In [15]:
with open('best_model.pkl', 'rb') as file:
    final_model = pickle.load(file)

print("Final model loaded successfully!")

Final model loaded successfully!


In [18]:
pipeline = Pipeline(steps=[
    ('drop_duplicates', DropDuplicatesTransformer()),
    ('missing_values', MissingValueHandler()),
    ('binning', BinningTransformer()),
    ('drop_columns', DropColumnsTransformer(columns_to_drop=['emp.var.rate', 'nr.employed', 'cons.price.idx', 'cons.conf.idx', 'campaign', 'pdays'])),
    ('model', final_model)
    ])

In [19]:
predictions = pipeline.predict(df)
predictions_df = pd.DataFrame(predictions, columns=["Predicted_y"])
result_df = pd.concat([df.reset_index(drop=True), predictions_df], axis=1)

[LightGBM] [Warning] min_data_in_leaf is set=20, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=20


/opt/anaconda3/lib/python3.12/site-packages/sklearn/preprocessing/_encoders.py:241: UserWarning: Found unknown categories in columns [1] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


<div style="text-align: left;">
  <h2 id="results" style="background-color: #003399; color: white; padding: 10px 20px; margin-bottom: 20px; border-radius: 8px; font-weight: bold; display: inline-block;"> 5. The Results </h2>
</div>

<div style='width: 100%; height: 0.05px; background-color: #003399; border-radius: 10px; margin-top: 20px; margin-bottom: 20px;'></div>

Here are the prediction results for the data that was previously input.


In [20]:
result_df

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,Predicted_y
0,38,student,married,basic.4y,yes,no,no,cellular,nov,mon,...,44,999,2,nonexistent,-3.383493,94.675099,-44.773086,1.771630,5228.1,1
1,50,admin.,unknown,professional.course,yes,yes,yes,telephone,jun,fri,...,24,999,1,nonexistent,1.387145,94.454781,-39.269555,4.326127,4963.6,1
2,31,technician,single,professional.course,no,unknown,yes,telephone,may,thu,...,11,999,6,nonexistent,-1.085762,93.327207,-32.444785,3.045997,4963.6,1
3,63,self-employed,married,professional.course,no,no,unknown,cellular,mar,tue,...,17,999,5,success,-1.814845,92.641923,-43.689510,3.226152,4963.6,1
4,66,management,single,basic.6y,yes,unknown,unknown,cellular,dec,tue,...,28,999,0,nonexistent,-2.501094,94.012726,-37.476266,4.899510,4963.6,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,56,blue-collar,single,basic.9y,no,no,yes,telephone,jul,thu,...,7,1,6,failure,-2.759690,92.876536,-37.399041,4.231601,4991.6,1
9996,66,services,single,basic.4y,no,unknown,yes,telephone,oct,thu,...,33,1,0,failure,0.191621,92.223093,-40.636561,0.711864,4991.6,1
9997,28,blue-collar,unknown,basic.4y,no,unknown,unknown,telephone,oct,tue,...,38,19,3,failure,-1.307462,93.518372,-40.654174,4.867393,5017.5,1
9998,33,technician,divorced,basic.9y,no,yes,yes,telephone,oct,mon,...,31,18,4,nonexistent,0.657740,94.286183,-27.570796,3.422417,5008.7,1


In [21]:
result_df[result_df["Predicted_y"] == 1]

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,Predicted_y
0,38,student,married,basic.4y,yes,no,no,cellular,nov,mon,...,44,999,2,nonexistent,-3.383493,94.675099,-44.773086,1.771630,5228.1,1
1,50,admin.,unknown,professional.course,yes,yes,yes,telephone,jun,fri,...,24,999,1,nonexistent,1.387145,94.454781,-39.269555,4.326127,4963.6,1
2,31,technician,single,professional.course,no,unknown,yes,telephone,may,thu,...,11,999,6,nonexistent,-1.085762,93.327207,-32.444785,3.045997,4963.6,1
3,63,self-employed,married,professional.course,no,no,unknown,cellular,mar,tue,...,17,999,5,success,-1.814845,92.641923,-43.689510,3.226152,4963.6,1
4,66,management,single,basic.6y,yes,unknown,unknown,cellular,dec,tue,...,28,999,0,nonexistent,-2.501094,94.012726,-37.476266,4.899510,4963.6,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,56,blue-collar,single,basic.9y,no,no,yes,telephone,jul,thu,...,7,1,6,failure,-2.759690,92.876536,-37.399041,4.231601,4991.6,1
9996,66,services,single,basic.4y,no,unknown,yes,telephone,oct,thu,...,33,1,0,failure,0.191621,92.223093,-40.636561,0.711864,4991.6,1
9997,28,blue-collar,unknown,basic.4y,no,unknown,unknown,telephone,oct,tue,...,38,19,3,failure,-1.307462,93.518372,-40.654174,4.867393,5017.5,1
9998,33,technician,divorced,basic.9y,no,yes,yes,telephone,oct,mon,...,31,18,4,nonexistent,0.657740,94.286183,-27.570796,3.422417,5008.7,1


In [22]:
result_df[result_df["Predicted_y"] == 0]

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,Predicted_y
36,76,management,unknown,university.degree,unknown,unknown,unknown,telephone,aug,tue,...,39,999,3,failure,-1.348325,93.519594,-40.606415,5.017737,5228.1,0
119,30,unemployed,unknown,university.degree,yes,no,no,telephone,jun,wed,...,9,999,4,failure,-1.112446,93.113694,-39.510334,3.691589,5008.7,0
121,32,housemaid,divorced,basic.6y,no,unknown,yes,cellular,jun,tue,...,30,999,3,nonexistent,-0.759935,92.598716,-50.061766,4.992168,5176.3,0
148,53,blue-collar,unknown,university.degree,no,yes,no,cellular,apr,tue,...,20,999,6,success,-0.740941,92.766971,-33.216985,4.126984,4963.6,0
151,27,management,unknown,basic.6y,no,no,yes,telephone,may,thu,...,7,999,3,failure,-3.292125,93.279497,-44.089621,3.756916,5228.1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9702,76,retired,divorced,illiterate,yes,unknown,unknown,telephone,aug,fri,...,3,21,6,failure,1.296861,92.936778,-46.529367,4.787156,5008.7,0
9882,64,blue-collar,single,basic.4y,yes,yes,yes,telephone,jun,wed,...,5,24,5,failure,-1.925043,94.562443,-42.975627,3.555880,5099.1,0
9892,58,housemaid,single,basic.6y,unknown,yes,yes,cellular,dec,mon,...,42,20,6,nonexistent,-1.961030,93.472280,-33.236513,3.684255,5023.5,0
9897,66,unemployed,unknown,university.degree,unknown,unknown,yes,telephone,may,wed,...,27,12,2,nonexistent,-3.140706,94.544898,-34.605533,3.690731,5017.5,0


<div style="text-align: center;">
  <h2 id="thanks" style="background-color: #003399; color: white; padding: 10px 20px; margin-bottom: 20px; border-radius: 10px; font-weight: bold;"> Thank You </h1>
</div>